In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['OMP_NUM_THREADS'] = '3'
os.environ['KMP_BLOCKTIME'] = '1'
import numpy as np
from tqdm.auto import tqdm
from dataset import get_labelled, get_unlabelled, _preload
from Commit import CommitFactory
from Model import CommitDiffModelFactory

In [2]:
CONTEXT_SIZE = 16
BAG_SIZE = 512
OUTPUT_SIZE = 128

In [3]:
Commit = CommitFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)
CommitDiffModel = CommitDiffModelFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE, OUTPUT_SIZE=OUTPUT_SIZE)

In [4]:
_preload(max_commit_bag_size = 4096, max_commits = 4096)

Loading Commit lookup table


  0%|          | 0/7 [00:00<?, ?it/s]

Loading file ../data/commit_lookups/supervised_commit_data_lookup0-1000.pickle
Appending pickle of length: 184 , new dict length: 126
Loading file ../data/commit_lookups/commit_data_lookup77500-80000.pickle
Appending pickle of length: 2500 , new dict length: 2407
Loading file ../data/commit_lookups/commit_data_lookup75000-77500.pickle


In [5]:
X_train, X_test, y_train, y_test = get_labelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

Generating Positive X_train:   0%|          | 0/97 [00:00<?, ?it/s]

Generating Positive y_train:   0%|          | 0/97 [00:00<?, ?it/s]

Generating Negative X_train:   0%|          | 0/70 [00:00<?, ?it/s]

Generating Negative y_train:   0%|          | 0/70 [00:00<?, ?it/s]

In [6]:
X_train_unsupervised = get_unlabelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

Generating Unsupervised X_train:   0%|          | 0/4096 [00:00<?, ?it/s]

In [7]:
print("Train set size", len(y_train))
print("Train set split", np.sum(y_train)/len(y_train))
print("Test set size", len(y_test))
print("Test set split", np.sum(y_test)/len(y_test))
print("Unsupervised Train Size", len(X_train_unsupervised))

Train set size 66
Train set split 0.5606060606060606
Test set size 17
Test set split 0.7058823529411765
Unsupervised Train Size 4096


In [ ]:
for encoder in tqdm(range(12)):
    try:
        
        model = CommitDiffModel(unsupervised_data_size = len(X_train_unsupervised))
        model.initialize(encoder=encoder)
        model.fit_siam(np.array(X_train_unsupervised), epochs=4, batch_size=512, verbose=1)
        model.fit_binary_classification(X_train, np.array(y_train), epochs=8, batch_size=8, verbose=1)
        score = model.evaluate_binary_classification(X_test, np.array(y_test), verbose=0)
        print("Enocder:", encoder)
        print("Score:", score)
    except Exception as e:
        print(e)
    print("------------------------------------------")

  0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1/4
8/8 [==============================] - 6s 288ms/step - loss: -0.1368
Epoch 2/4
8/8 [==============================] - 2s 298ms/step - loss: -0.3854
Epoch 3/4
8/8 [==============================] - 2s 282ms/step - loss: -0.4951
Epoch 4/4
8/8 [==============================] - 2s 267ms/step - loss: -0.5298
Epoch 1/8
9/9 [==============================] - 4s 28ms/step - loss: 0.7920
Epoch 2/8
9/9 [==============================] - 0s 24ms/step - loss: 0.7302
Epoch 3/8
9/9 [==============================] - 0s 30ms/step - loss: 0.6906
Epoch 4/8
9/9 [==============================] - 0s 26ms/step - loss: 0.7382
Epoch 5/8
9/9 [==============================] - 0s 26ms/step - loss: 0.7434
Epoch 6/8
9/9 [==============================] - 0s 25ms/step - loss: 0.7159
Epoch 7/8
9/9 [==============================] - 0s 27ms/step - loss: 0.6779
Epoch 8/8
9/9 [==============================] - 0s 25ms/step - loss: 0.6665
Enocder: 0
Score: 0.6251628994941711
-------------------------------